In [26]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf
import h5py
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from tqdm import tqdm
from pathlib import Path
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import callbacks
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GaussianNoise
from keras import regularizers
from download_h5py_db import download_h5py_db
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from pathlib import Path
from utils import show_results
from h5py import File
import tensorflow as tf


c:\Users\simon\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\client\session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [27]:
FILE_NAME = 'test_data.h5'
path = 'res//res.h5'
SIZE=224
num_classes = 5

Helper Functions

In [28]:
def font_to_num(font):
    if font == b'Alex Brush':
        return 0
    elif font == b'Open Sans':
        return 1
    elif font == b'Sansation':
        return 2
    elif font == b'Titillium Web':
        return 3
    else:
        return 4

In [29]:
def num_to_font(font):
    if font == 0:
        return b'Alex Brush'
    elif font == 1:
        return b'Open Sans'
    elif font == 2:
        return b'Sansation'
    elif font == 3:
        return b'Titillium Web'
    else:
        return b'Ubuntu Mono'

In [30]:

def prepare_img(img, bbs, index, size = SIZE):
    x1 = int(bbs[0,0,index])
    y1 = int(bbs[1,0,index])
    x2 = int(bbs[0,1,index])
    y2 = int(bbs[1,1,index])
    x3 = int(bbs[0,2,index])
    y3 = int(bbs[1,2,index])
    x4 = int(bbs[0,3,index])
    y4 = int(bbs[1,3,index])
    # calculate bounding rectangle
    top_left_x = max(0, min([x1,x2,x3,x4]))
    top_left_y = max(0, min([y1,y2,y3,y4]))
    bot_right_x = max(0, max([x1,x2,x3,x4]))
    bot_right_y = max(0, max([y1,y2,y3,y4]))

    cropped = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    cropped = tf.image.resize(cropped, (size, size), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    cropped = tf.image.convert_image_dtype(cropped, tf.float32)
    return cropped

In [31]:

def create_dirs(main_dir):
    Path(main_dir).mkdir(parents=True, exist_ok=True)
    Path(main_dir+'/Alex Brush').mkdir(parents=True, exist_ok=True)
    Path(main_dir+'/Titillium Web').mkdir(parents=True, exist_ok=True)
    Path(main_dir+'/Sansation').mkdir(parents=True, exist_ok=True)
    Path(main_dir+'/Open Sans').mkdir(parents=True, exist_ok=True)
    Path(main_dir+'/Ubuntu Mono').mkdir(parents=True, exist_ok=True)

In [32]:
def get_image_data(db, im, has_labels=True):
    img  = db['data'][im][:]
    if has_labels:
        fonts = db['data'][im].attrs['font']  
    else: 
        fonts = None
    txts = db['data'][im].attrs['txt']
    charBBs = db['data'][im].attrs['charBB']
    wordBBs = db['data'][im].attrs['wordBB']
    return img, fonts, txts, charBBs, wordBBs

In [33]:
def is_not_dot(inp):
    res= (inp != ord('.') and inp != ord(':'))
    return res

In [34]:
def crop_and_save(img, BBs, indx, size, curr_font, im, num, append_not_save=False, folder='main_directory/'):
    cropped = prepare_img(img, BBs, indx, size)
    if not append_not_save:
        path = folder+curr_font.decode('UTF-8')+'/'+im+'_'+str(num)+'.png'
        tf.keras.utils.save_img(path,cropped)
    return cropped

In [35]:
from sklearn.preprocessing import normalize
from utils import create_dirs, SIZE, get_image_data, crop_and_save, font_to_num
from numpy import asarray, argmax, bincount,argwhere
from h5py import File
from sklearn.preprocessing import normalize
import csv
from h5py import File
from tqdm import tqdm
import numpy as np
from keras.utils import np_utils


class Font_classifier:
    def __init__(self, model):
        self.model = model

    def make_row(self, index, im, ch, prediction, writer):
        a = []
        a.append(index)
        a.append(im)
        a.append(ch)
        b = np_utils.to_categorical(prediction, 5)
        # print(prediction)
        # print(a)
        # print(b)
        b = np.concatenate((a,b[1:],[b[0]]))
        writer.writerow(b)
        index+=1

    def predict(self, db_file):
        db = File(db_file, 'r')   
        size=SIZE
        im_names = list(db['data'].keys())
        num = 0
        index = 0
        # prediction_arr=[]
        # images = []
        # chars=[]
        with open('test_lables.csv', 'w+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([" ", "image", "char","Open Sans","Sansation","Titillium Web","Ubuntu Mono","Alex Brush"])
            for i in tqdm(range(0, len(im_names))):
                im = im_names[i]
                img, _, txts, charBBs, wordBBs = get_image_data(db, im, has_labels=False)
                font_indx = 0 
                char_indx = 0
                for j in range(0, len(txts)):
                    test_x = [] 
                    cropped = crop_and_save(img, wordBBs, j, size, None, im, num, True)
                    test_x.append(cropped)
                    num+=1            
                    for k in range(0, len(txts[j])):
                        word =  txts[j]
                        cropped = crop_and_save(img, charBBs, char_indx, size, None, im, num, True)
                        test_x.append(cropped)
                        num+=1
                        # char_indx+=1
                        index += 1
                    test_x = np.asarray(test_x, dtype=np.float32)
                    reses = self.model.predict(test_x, verbose=0)
                    maxes = np.argmax(reses, axis=1)
                    prediction = np.bincount(maxes)
                    prediction = np.argwhere(prediction==prediction.max())
                    if (len(prediction)>1):
                        reses_n = np.normalize(reses, axis=1, norm='l1')
                        maxes_n = np.argmax(reses_n, axis=1)
                        prediction = np.bincount(maxes_n)
                        prediction = np.argwhere(prediction==prediction.max())
                        if(len(prediction)>1):
                            sum_p = np.sum(reses, axis=0)
                            prediction = sum_p.argmax()
                    for k in range(0, len(word)):
                        a = []
                        a.append(index)
                        a.append(im)
                        a.append(chr(word[k]))
                        b = np_utils.to_categorical(prediction.item(), 5)
                        b = np.concatenate((a,b[1:],[b[0]]))
                        # print(b)
                        writer.writerow(b)
                        index+=1
                        # prediction_arr.append(prediction.item())
                        # images.append(im)
                    font_indx += len(txts[j])



In [36]:
model = load_model(path)

In [37]:
FILE_NAME = 'test_data.h5'
if not Path('test_data.h5').exists():
    download_file_from_google_drive('1YwLcXqLArFSOtoepQw7nC1t4jC8CFxpI', FILE_NAME)


In [38]:
def predict(model_, db_file):
        db = File(db_file, 'r')   
        size=SIZE
        im_names = list(db['data'].keys())
        num = 0
        index = 0
        # prediction_arr=[]
        # images = []
        # chars=[]
        with open('test_lables.csv', 'w+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([" ", "image", "char","Open Sans","Sansation","Titillium Web","Ubuntu Mono","Alex Brush"])
            for i in tqdm(range(0, len(im_names))):
                im = im_names[i]
                img, _, txts, charBBs, wordBBs = get_image_data(db, im, has_labels=False)
                font_indx = 0 
                char_indx = 0
                for j in range(0, len(txts)):
                    test_x = [] 
                    cropped = crop_and_save(img, wordBBs, j, size, None, im, num, True)
                    test_x.append(cropped)
                    num+=1            
                    for k in range(0, len(txts[j])):
                        word =  txts[j]
                        cropped = crop_and_save(img, charBBs, char_indx, size, None, im, num, True)
                        test_x.append(cropped)
                        num+=1
                        # char_indx+=1
                        index += 1
                    test_x = np.asarray(test_x, dtype=np.float32)
                    reses = model_.predict(test_x, verbose=0)
                    maxes = np.argmax(reses, axis=1)
                    prediction = np.bincount(maxes)
                    prediction = np.argwhere(prediction==prediction.max())
                    if (len(prediction)>1):
                        reses_n = np.normalize(reses, axis=1, norm='l1')
                        maxes_n = np.argmax(reses_n, axis=1)
                        prediction = np.bincount(maxes_n)
                        prediction = np.argwhere(prediction==prediction.max())
                        if(len(prediction)>1):
                            sum_p = np.sum(reses, axis=0)
                            prediction = sum_p.argmax()
                    for k in range(0, len(word)):
                        a = []
                        a.append(index)
                        a.append(im)
                        a.append(chr(word[k]))
                        b = np_utils.to_categorical(prediction.item(), 5)
                        b = np.concatenate((a,b[1:],[b[0]]))
                        # print(b)
                        writer.writerow(b)
                        index+=1
                        # prediction_arr.append(prediction.item())
                        # images.append(im)
                    font_indx += len(txts[j])

In [39]:
pred_y = predict(model, FILE_NAME)


  0%|          | 5/1004 [04:05<13:36:27, 49.04s/it]


KeyboardInterrupt: 